In [ ]:
# Importing packages
import netCDF4
from netCDF4 import Dataset
import numpy as np
import datetime


def make_ncfile(data_grid, x_coord, y_coord, lat_grid, lon_grid, start_year, fname):
   
    # Associated time:
    dyr = 2
    center_yr = start_year + dyr/2
   
    # Use the following reference date:
    reference_date = datetime.datetime(2010,1,1)
   
    with (netCDF4.Dataset(fname, 'w', format='NETCDF4_CLASSIC')) as ncout:
        # Create dimensions
        yn, xn = np.shape(data_grid)
        # Recommended to be in the following order: T, Y, X
        dim_time = ncout.createDimension('time',1) # None? (if unlimited)
        dim_y = ncout.createDimension('y',yn)
        dim_x = ncout.createDimension('x',xn)
        # And a dimension for the time bounds of each time period
        dim_bounds = ncout.createDimension('bnds',2)
   
        # Data for the time dimension:
        nctime = ncout.createVariable('time','i4',('time',))
        # Add time axis data:
        # Convert to format: no of days since the reference date
        center_date = datetime.datetime(int(center_yr),1,1)
        days_since_reference = (center_date - reference_date).days
        nctime[0] = days_since_reference
        # Set name of variable containing associated bounds of time axis (constructed below):
        nctime.bounds = 'time_bounds'
        # Add meta data for time axis:
        nctime.standard_name = 'time'
        nctime.long_name = 'Mid-point time of acquisitions used for inversion'
        nctime.units = f'days since {reference_date.strftime("%Y-%m-%d")}'
   
        # Set variable for associated bounds of the time axis:
        nctimebounds = ncout.createVariable('time_bounds','i4',('time','bnds'))
        start_date = datetime.datetime(int(center_yr - dyr/2),1,1)
        end_date = datetime.datetime(int(center_yr + dyr/2),1,1)-datetime.timedelta(days=1)
        days_since_reference_start = (start_date - reference_date).days
        days_since_reference_end = (end_date - reference_date).days
        nctimebounds[0,0] = days_since_reference_start
        nctimebounds[0,1] = days_since_reference_end
        nctimebounds.units =  f'days since {reference_date.strftime("%Y-%m-%d")}'
       
        # Creating variables deciding the extent of the product in the coordinate reference system
        # Data for the x-axis:
        ncx = ncout.createVariable('x','i4', ('x',))
        ncx[:] = x_coord
        # Add meta data:
        ncx.standard_name= 'projection_x_coordinate'
        ncx.long_name= 'x coordinate in projected coordinate system'
        ncx.units = 'm'
        #ncx.axis = 'X' # optional
       
        # Data for the y-axis:
        ncy = ncout.createVariable('y','i4', ('y',))
        ncy[:] = y_coord
        # Add meta data:
        ncy.standard_name= 'projection_y_coordinate'
        ncy.long_name= 'x coordinate in projected coordinate system'
        ncy.units = 'm'
        #ncy.axis = 'Y' # optional
           
        # Add lon, lat variables as auxilliary coordinates:
        nclat = ncout.createVariable('lat', 'f4', ('y','x'))
        nclon = ncout.createVariable('lon', 'f4', ('y','x'))
        # Add data for gridded values of lat, lon:
        nclat[:,:] = lat_grid
        nclon[:,:] = lon_grid
        # Add meta data:
        nclat.standard_name = 'latitude'
        nclat.long_name = 'latitude'
        nclat.units = 'degrees_north'
        nclat.grid_mapping = 'UTM_projection'
        nclon.standard_name = 'longitude'
        nclon.long_name = 'longitude'
        nclon.units = 'degrees_east'
        nclon.grid_mapping = 'UTM_projection'
         
        # Add GRACE inversion data variable:
        ncjointinv = ncout.createVariable('annual_mass_change', 'f8', ('time', 'y', 'x'), zlib=True)
        # Add data:
        ncjointinv[0,:,:] = data_grid # OBS: Using 0 pga. tidslag 1
        # Add meta-data:
        ncjointinv.units = 'mm_per_year' # OBS: in water equivalents!
        ncjointinv.comment = "Non-standard unit"
        ncjointinv.standard_name = 'tendency_of_land_ice_mass' # unit: kg/s???
        ncjointinv.long_name = 'annual mass change (2 yr mean) in mm water equivalents'
        # Option to add more info!
        ncjointinv.missing_value = np.nan
        ncjointinv.cell_methods = "mean" # correct?
        # evt: time: sum over years
        # precipitation:cell_methods="time: sum within years time: mean over years";  decadal averages for january
       
        # Adding coordinate reference to the variables
        ncjointinv.coordinates = "lon lat"
        ncjointinv.grid_mapping = 'UTM_projection'
   
        # Creating coordinate reference variable (attributes depends on projection)
        nc_crs = ncout.createVariable('UTM_projection',np.int32, ())
        nc_crs.grid_mapping_name = 'transverse_mercator'
        # Required parameters for this projection:
        nc_crs.latitude_of_projection_origin = 0.0
        nc_crs.scale_factor_at_central_meridian = 0.9996
        nc_crs.longitude_of_central_meridian = -45
        nc_crs.false_easting = 500000.0
        nc_crs.false_northing = 0.0
        # Additional info:
        nc_crs.proj4_string = "+proj=utm +zone=23 +datum=WGS84 +units=m +no_defs"
        nc_crs.epsg_code = 32623

    #%% Add global attributes:
    nowstr = datetime.date.today().isoformat() # current date
   
    # Option dataset:
    ncout = Dataset(fname, mode='r+') # r+ for append mode
   
    startdate_str = start_date.strftime("%Y-%m-%d")
    enddate_str = end_date.strftime("%Y-%m-%d")
   
    globalAttribs = {}
    globalAttribs['title'] = "Greenland Ice Sheet annual mass changes from joint inversion of gravimetry and altimetry"    
    globalAttribs['summary'] = 'Greenland Ice Sheet annual mass changes from ' + startdate_str + " to " + enddate_str
    globalAttribs['Conventions'] = "CF-1.6"
   
    globalAttribs['time_coverage_start'] = startdate_str
    globalAttribs['time_coverage_end'] = enddate_str
   
    globalAttribs['institution'] = "Technical University of Denmark (DTU)"
    globalAttribs['project'] = 'PROMICE'
    globalAttribs['date_created'] = nowstr
    globalAttribs['source'] =  "Remote sensing of gravimetry (GRACE, GRACE-FO) and altimetry (CryoSat-2)"
    #globalAttribs['references'] =  "give a ref" # OBS - what to give here?
   
   
    ncout.setncatts(globalAttribs)
    ncout.sync()
   
    ncout.close()
   
    return

In [43]:
%cd C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor

# imports
import pickle
import os
from pathlib import Path

import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from cartopy import feature as cfeature
from pyproj import Transformer

from src.utils.data_utils import DataMapping
import src.utils.standard_plots as sp

C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor


In [2]:
PREDICTIONS_FOLDER = 'data/processed/predictions/'
CETB_FOLDER = 'data/raw/CETB_AMSR2/'

In [21]:
CETB_mapping = DataMapping(CETB_FOLDER,'CETB')

# map SD data
sd_files = os.listdir(PREDICTIONS_FOLDER)

sd_df = pd.DataFrame({'date':[], 'filename': []})
for file in sd_files:
    file = Path(os.path.join(PREDICTIONS_FOLDER, file))
    stem = file.stem
    sd_df.loc[len(sd_df)] = [stem.split('_')[-1], file]

files = sd_df['filename'].tolist()[:4]
ds_main = xr.open_mfdataset(files[0])
ds_main

<xarray.Dataset> Size: 66MB
Dimensions:  (time: 1, y: 2880, x: 2880)
Coordinates:
  * time     (time) datetime64[ns] 8B 2020-11-01
  * y        (y) float64 23kB 4.498e+06 4.495e+06 ... -4.495e+06 -4.498e+06
  * x        (x) float64 23kB -4.498e+06 -4.495e+06 ... 4.495e+06 4.498e+06
Data variables:
    SD_mean  (time, y, x) float64 66MB dask.array<chunksize=(1, 2880, 2880), meta=np.ndarray>

In [44]:
trf = Transformer.from_crs("EPSG:6931", "EPSG:4326", always_xy=True)
lat, lon = trf.transform(ds_main['x'].values, ds_main['y'].values)
lat_grid, lon_grid = np.meshgrid(lat, lon)

In [53]:
from src.utils.grid_utils import Grid
grid = Grid.from_predefined('EASE2_N3.125km')
grid.modify_extent([-4500000, -4500000, 4500000, 4500000])

In [59]:
from pyproj import Proj
proj = Proj("EPSG:6931")
proj.to_proj4()

'+proj=laea +lat_0=90 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'

In [67]:
proj.crs.area_of_use

AreaOfUse(west=-180.0, south=0.0, east=180.0, north=90.0, name='Northern hemisphere.')

In [65]:
proj.crs

<Projected CRS: EPSG:6931>
Name: WGS 84 / NSIDC EASE-Grid 2.0 North
Axis Info [cartesian]:
- X[south]: Easting (metre)
- Y[south]: Northing (metre)
Area of Use:
- name: Northern hemisphere.
- bounds: (-180.0, 0.0, 180.0, 90.0)
Coordinate Operation:
- name: US NSIDC EASE-Grid 2.0 North
- method: Lambert Azimuthal Equal Area
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [77]:
from datetime import datetime

In [ ]:
x_attrs = {
    'standard_name': 'projection_x_coordinate',
    'coverage_content_type': 'coordinate',
    'long_name': 'x',
    'units': 'meters',
    'axis': 'X',
    'valid_range': [-9000000.0, 9000000.0]
}
y_attrs = {
    'standard_name': 'projection_y_coordinate',
    'coverage_content_type': 'coordinate',
    'long_name': 'y',
    'units': 'meters',
    'axis': 'Y',
    'valid_range': [-9000000.0, 9000000.0]
}
lat_attrs = {
    'standard_name': 'latitude',
    'long_name': 'latitude',
    'units': 'degrees_north',
    'valid_range': [-90.0, 90.0],
}
lon_attrs = {
    'standard_name': 'longitude',
    'long_name': 'longitude',
    'units': 'degrees_east',
    'valid_range': [-180.0, 180.0],
}
time_attrs = {
    'standard_name': 'time',
    'coverage_content_type': 'coordinate',
    'long_name': 'time',
    'axis': 'T',
    'units': 'seconds since 1970-01-01T00:00:00Z',
}
sd_attrs = {
    'standard_name': 'snow_depth',
    'long_name': 'Snow Depth',
    'description': 'Snow Depth data derived using model utilizing inputs AMSR2 measurements from NSIDC-0630 (CETB) and ERA5 daily reanalysis data, and trained using CRYO2ICE (C2I) data from doi:10.1029/2023EA003313',
    'units': 'meters',
    'valid_range': [0.0, 10.0],
    'description': 'Snow Depth data',
    'unit': 'meters',
    'grid_mapping' : 'crs'
}
crs_attrs = {
    'grid_mapping_name': 'lambert_azimuthal_equal_area',
    'longitude_of_projection_origin': 0.0,
    'latitude_of_projection_origin': 90.0,
    'false_easting': 0.0,
    'false_northing': 0.0,
    'semi_major_axis': 6378137.0,
    'inverse_flattening': 298.257223563,
    'proj4text': '+proj=laea +lat_0=90 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m',
    'srid': 'urn:ogc:def:crs:EPSG::6931',
    'coverage_content_type' : 'auxiliaryInformation',
    'references' : ["EASE-Grid 2.0 documentation:  https://nsidc.org/data/user-resources/help-center/guide-ease-grids", "Brodzik, Mary J.; Billingsley, Brendan; Haran, Terry; Raup, Bruce; Savoie, Matthew H. 2012.", "EASE-Grid 2.0: Incremental but Significant Improvements for Earth-Gridded Data Sets.", "ISPRS Int. J. Geo-Inf. 1, no. 1: 32-45.", "https://doi:10.3390/ijgi1010032", "Brodzik, Mary J.; Billingsley, Brendan; Haran, Terry; Raup, Bruce; Savoie, Matthew H. 2014.", "Correction: Brodzik, M. J., et al. EASE-Grid 2.0: Incremental but Significant Improvements for Earth-Gridded Data Sets.", "ISPRS Int. J. Geo-Inf. 3, no. 3: 1154-1156.", "https://doi:10.3390/ijgi3031154"],
    'crs_wkt' : 'PROJCRS["WGS 84 / NSIDC EASE-Grid 2.0 North", BASEGEODCRS["WGS 84", DATUM["World Geodetic System 1984", ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1.0]]]], CONVERSION["US NSIDC EASE-Grid 2.0 North", METHOD["Lambert Azimuthal Equal Area",ID["EPSG",9820]], PARAMETER["Latitude of natural origin",90,ANGLEUNIT["degree",0.01745329252]], PARAMETER["Longitude of natural origin",0,ANGLEUNIT["degree",0.01745329252]], PARAMETER["False easting",0,LENGTHUNIT["metre",1.0]], PARAMETER["False northing",0,LENGTHUNIT["metre",1.0]]], CS[cartesian,2], AXIS["easting (X)",south,MERIDIAN[90,ANGLEUNIT["degree",0.01745329252]],ORDER[1]], AXIS["northing (Y)",south,MERIDIAN[180,ANGLEUNIT["degree",0.01745329252]],ORDER[2]], LENGTHUNIT["metre",1.0], ID["EPSG",6931]]',
    'long_name' : 'EASE2_N3.125km',
    'GeoTransform' : '-4500000.00000  3125.00000 0.00000 4500000.00000 0.00000 -3125.00000'
}
attrs = {
    'title': 'Automatic Snow Products - Snow Depth V1.0',
    'summary': 'This dataset contains snow depth (SD) estimates over the Arctic region derived from AMSR2 brightness temperature measurements and ERA5 reanalysis data using a machine learning model (Random Forest Regression) trained on snow depth observations from the CRYO2ICE campaign during the winter season 2020-2021 and 2021-2022.',
    'contributer_name': 'Emil H. Tellefsen, Mai Winstrup, Henriette Skourup, Renée M. Fredensborg Hansen',
    'contributer_role': 'principal_investigator, co_investigator, co_investigator, co_investigator',
    'software_repository': 'https://github.com/EHTellefsen/Automatic_Snow_Products',
    'data_sources': 'CETB AMSR2 NSIDC-0630 (https://nsidc.org/data/nsidc-0630/versions/2), \nERA5 reanalysis (https://cds.climate.copernicus.eu/datasets/derived-era5-single-levels-daily-statistics?tab=overview), \nCRYO2ICE snow depth data (https://data.dtu.dk/articles/dataset/CRYO2ICE_radar_laser_freeboards_snow_depth_on_sea_ice_and_comparison_against_auxiliary_data_during_winter_season_2020-2021/21369129)',
    'Conventions': 'CF-1.9, ACDD-1.3',
    'institution': 'DTU Space',
    'geospatial_lat_min': 0.0,
    'geospatial_lat_max': 90.0,
    'geospatial_lon_min': -180.0,
    'geospatial_lon_max': 180.0,
    'geospatial_lat_units': 'degrees_north',
    'geospatial_lon_units': 'degrees_east',
    'email': 'maiwin@space.dtu.dk',
    'date_created': datetime.today().strftime('%Y-%m-%d')
}

ds = ds_main.rename({'SD_mean':'sd'})
# convert time coordinate to seconds precision using assign_coords (avoid attribute assignment on Dataset)
ds = ds.assign_coords(x=ds['x'].astype('float32'))
ds = ds.assign_coords(y=ds['y'].astype('float32'))
ds = ds.assign_coords(lat=(('y','x'), lat_grid.astype('float32')))
ds = ds.assign_coords(lon=(('y','x'), lon_grid.astype('float32')))
ds['sd'] = ds.sd.astype('float32')
#ds = ds.assign_coords(sd=ds['sd'].astype('float32'))
ds['time'].attrs = time_attrs
ds['x'].attrs = x_attrs
ds['y'].attrs = y_attrs
ds['lat'].attrs = lat_attrs
ds['lon'].attrs = lon_attrs
ds['sd'].attrs = sd_attrs
# Ensure 'crs' coordinate/variable exists before setting attributes
ds['crs'] = xr.DataArray(np.int32(0))
ds['crs'].attrs = crs_attrs
ds = ds.chunk({'time': 1, 'y': 720, 'x': 720})
ds.attrs = attrs
ds

<xarray.Dataset> Size: 100MB
Dimensions:  (time: 1, y: 2880, x: 2880)
Coordinates:
  * time     (time) datetime64[ns] 8B 2020-11-01
  * x        (x) float32 12kB -4.498e+06 -4.495e+06 ... 4.495e+06 4.498e+06
  * y        (y) float32 12kB 4.498e+06 4.495e+06 ... -4.495e+06 -4.498e+06
    lat      (y, x) float32 33MB dask.array<chunksize=(720, 720), meta=np.ndarray>
    lon      (y, x) float32 33MB dask.array<chunksize=(720, 720), meta=np.ndarray>
Data variables:
    sd       (time, y, x) float32 33MB dask.array<chunksize=(1, 720, 720), meta=np.ndarray>
    crs      int32 4B 0
Attributes: (12/16)
    title:                 Automatic Snow Products - Snow Depth V1.0
    summary:               This dataset contains snow depth (SD) estimates ov...
    contributer_name:      Emil H. Tellefsen, Mai Winstrup, Henriette Skourup...
    contributer_role:      principal_investigator, co_investigator, co_invest...
    software_repository:   https://github.com/EHTellefsen/Automatic_Snow_Prod...
    data_sources:          CETB AMSR2 NSIDC-0630 (https://nsidc.org/data/nsid...
    ...                    ...
    geospatial_lon_min:    -180.0
    geospatial_lon_max:    180.0
    geospatial_lat_units:  degrees_north
    geospatial_lon_units:  degrees_east
    email:                 maiwin@space.dtu.dk
    date_created:          2025-10-27

In [ ]:
ds.to_netcdf('data/processed/predictions/SD_20220101.nc',)

In [86]:
list(ds.data_vars)

['sd', 'crs']

In [80]:
datetime.today().strftime('%Y-%m-%d')

'2025-10-27'

In [68]:
proj.crs.area_of_use.bounds

(-180.0, 0.0, 180.0, 90.0)

In [36]:
ds.chunk({'time': 1, 'y': 720, 'x': 720})

<xarray.DataArray 'sd' (time: 1, y: 2880, x: 2880)> Size: 33MB
dask.array<astype, shape=(1, 2880, 2880), dtype=float32, chunksize=(1, 2880, 2880), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[s] 8B 2020-11-01
  * x        (x) int32 12kB -4498437 -4495312 -4492187 ... 4495312 4498437
  * y        (y) int32 12kB 4498437 4495312 4492187 ... -4495312 -4498437

In [27]:
ds.time

<xarray.DataArray 'time' (time: 1)> Size: 8B
array(['2020-11-01T00:00:00'], dtype='datetime64[s]')
Coordinates:
  * time     (time) datetime64[s] 8B 2020-11-01
    crs      int32 4B 0
Attributes:
    standard_name:  time
    long_name:      time
    units:          nanoseconds since 1970-01-01T00:00:00Z